In [1]:
import numpy as np
from qiskit import(
  QuantumCircuit,
  execute,
  BasicAer,
  QuantumRegister,
  ClassicalRegister
)
from qiskit.visualization import plot_histogram

In [2]:
def gen_measurements(circuit, backend, shots):
    N = len(circuit.qubits)
    res = []
    # z measurement
    circ = circuit.copy()
    circ.measure([x for x in range(N)],[x for x in range(N)])
    result = execute(circ, backend=backend, shots=shots).result().get_counts()
    for x in range(2**N):
        key = '{number:0{width}b}'.format(width=N,number=x)
        if key not in result:
            result[key] = 0      
    res.append(result)
    # one-x measurements
    for i in range(N):
        circ = circuit.copy()
        circ.h(i)
        circ.measure([x for x in range(N)],[x for x in range(N)])
        result = execute(circ, backend=backend, shots=shots).result().get_counts()
        for x in range(2**N):
            key = '{number:0{width}b}'.format(width=N,number=x)
            if key not in result:
                result[key] = 0      
        res.append(result)
    return res

def estimate_vector(measurements, dim):
    #absolute values estimation
    meas = measurements[0]
    estvect = [np.sqrt(meas.get('{number:0{width}b}'.format(width=dim,number=i))/sum(meas.values())) for i in range(2**dim)]
    #signs 
    for i in range(dim):
        estvect = np.reshape(estvect,(-1,2**i))
        estvect = rel_sign(estvect, measurements[i+1], i)
    estvect = np.reshape(estvect,(-1,2**dim))
    return estvect

def rel_sign(vect, measurement, n):
    meas = sorted(measurement.items())
    for i in range(0,len(vect),2):
            if meas[i*(2**n)][1] < meas[(i+1)*(2**n)][1]:
                vect[i+1] *= -1
    return vect

In [6]:
vect = np.random.rand(64)*2-1
vect = vect/np.linalg.norm(vect)
print("Real vector:",vect)

simulator = BasicAer.get_backend('qasm_simulator')

circuit = QuantumCircuit(6, 6)
circuit.initialize(vect,circuit.qubits)
measurements = gen_measurements(circuit, simulator, 8000)
vector = estimate_vector(measurements, len(circuit.qubits))

print("Estimated:",vector)

Real vector: [ 0.0133037   0.17787972  0.05143998  0.01534008  0.17243966  0.09538567
 -0.18361977 -0.2082282   0.06490131 -0.18212693  0.10907535  0.06379422
 -0.02159684  0.0717469  -0.03967729  0.11846587  0.03408098 -0.04821098
 -0.10782044 -0.1745602  -0.07479563  0.18399008  0.18784807 -0.0173821
  0.17363766  0.18032631 -0.11910093 -0.21857194  0.13004034 -0.12281693
 -0.21036158  0.00128238 -0.15441032 -0.14331005  0.03338858  0.05408904
  0.11614812 -0.05539334 -0.13501242  0.00730328  0.04753614  0.04489713
  0.15651751 -0.21960503  0.1194423   0.1575945  -0.13462875  0.19103362
  0.05542984 -0.0588734  -0.00795047  0.18196389  0.06881131  0.12705652
  0.02255045  0.1394396  -0.1559745   0.08303021 -0.04300533 -0.19271931
  0.01893318 -0.03602596 -0.13425481 -0.17002628]
Estimated: [[ 0.          0.18973666  0.05244044  0.01118034  0.17923448  0.09552487
  -0.18506756 -0.20248457  0.07158911 -0.18027756  0.11779219  0.075
  -0.01581139  0.07331439 -0.04330127  0.11510864  0.0